# Importação das bibliotecas necessárias

In [1]:
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import  classification_report
from sklearn.feature_selection import SelectPercentile

In [2]:
#imports
from time import time
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC
#from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import optuna
from sklearn.metrics import f1_score

In [3]:

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
import nltk
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import RFECV, RFE, SelectKBest
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn import set_config
set_config(display='diagram')
from sklearn.svm import SVC
import datetime

# Importação dos dados

In [4]:
data = pd.read_csv('../../../../data/processed/(CORRIGIDO)ep2_pln_train_pos_pt_core_news_lg.xlsx')
data.drop(['Unnamed: 0', 'req_text'], axis=1, inplace=True)

In [5]:
data.head() # visualização das primeiras 5 linhas do dataframe

,age,pos
0,a2,PUNCT DET NOUN ADP DET NUM ADP ADJ NOUN ADJ PU...
1,a2,PUNCT VERB NOUN ADP PROPN ADP PROPN ADP NOUN A...
2,a2,PUNCT VERB VERB DET NOUN PUNCT NOUN PUNCT NOUN...
3,a2,PUNCT VERB ADP NOUN PUNCT DET NOUN ADP DET NOU...
4,a2,PUNCT VERB ADP NOUN PUNCT DET NOUN ADP DET NOU...


In [6]:
data.shape # visualização do formato do dataframe

(8200, 2)

In [7]:
data.age.value_counts()

age
a3    3000
a2    2000
a4    2000
a1    1200
Name: count, dtype: int64

In [8]:
data['age'] = data['age'].map({
                                'a4':3,
                                'a3': 2,
                                'a2': 1,
                                'a1': 0})

In [9]:
data.head()

,age,pos
0,1,PUNCT DET NOUN ADP DET NUM ADP ADJ NOUN ADJ PU...
1,1,PUNCT VERB NOUN ADP PROPN ADP PROPN ADP NOUN A...
2,1,PUNCT VERB VERB DET NOUN PUNCT NOUN PUNCT NOUN...
3,1,PUNCT VERB ADP NOUN PUNCT DET NOUN ADP DET NOU...
4,1,PUNCT VERB ADP NOUN PUNCT DET NOUN ADP DET NOU...


In [10]:
# a seguir os dados serão divididos entre features (X) e label (y)
X = data['pos'] # features
y = data['age'] # label

# Seleção de hiperparâmetros

Nesta seção, será feita a comparação entre classificadores com os melhores hiperparâmetros encontrados através da utilização do otimizador Optuna, que, resumidamente, funciona de X forma. Os classificadores comparados são:
- Regressão Logística
- Random Forest
- XGBoost

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,
                                                   stratify=y)

# Seleção do modelo

In [13]:
def seleciona_grid(model):

    param_grid = None

    if isinstance(model, MultinomialNB):
            param_grid = {
            "vect__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
            "selection__percentile": [33, 66, 100],
            "estimator__alpha": [50, 15, 10, 5, 1, 0.5, 0.3, 0.1, 0.05, 0.03, 0.02, 0.01,  0.001],
            "estimator__fit_prior": [True, False],
            }

    return param_grid

In [14]:
def fit_e_avalia(random_search):
    
    inicio_random_search = datetime.datetime.now()

    model_trained = random_search.fit(X_train, y_train) # fit

    fim_random_search = datetime.datetime.now()
    tempo_total = fim_random_search - inicio_random_search
    print(f'Duração da Random Search: {tempo_total}')

    y_pred = model_trained.predict(X_test) # predicao

    # Predição F1 e Class Report
    f1 = f1_score(y_test, y_pred, average= 'macro') # f1
    f1 *= 100
    f1 = round(f1,2)
    report = classification_report(y_test, y_pred, output_dict=True) # class report
    
    return model_trained, tempo_total, f1, report

In [15]:
def compara(iteracoes, modelos, nome_arquivo):

    # seletor de features
    selection = SelectPercentile()

    # possibilidades de oversampling ou nao
    #samplers = [RandomOverSampler(random_state=42), None]

    # dataframe em que sera inserido os dados do modelo testado 
    df_resultados = pd.DataFrame(columns=['modelo', 'vect', 'scaling',
                                          'duracao_random_search','qnt_iteracoes',
                                          'f1_randsearch', 
                                           'f1_pred',
                                          'class_report',
                                         'duracao_aval_iv',
                                          'acc_aval_iv',
                                         'melhores_parametros'])
    
    for model in modelos:

                
        # seleciona grid de parametros
        param_grid = seleciona_grid(model)
        
        
        scaler = MaxAbsScaler()

        # define o pipeline
        pipeline = Pipeline([
                ('vect', TfidfVectorizer(analyzer='word')),
                ('scaling', scaler), 
                ('selection', selection),
                ('estimator', model)
                ])

        
        #  --- Prints das configurações dessa iteracao ---
        print(f'Modelo: {model}')
        print(f'Scaler: {scaler}')
            

        # definicao da randomized search
        random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid,cv=StratifiedKFold(n_splits=5),
                                            n_iter=iteracoes, n_jobs=1, random_state=42, scoring='f1_macro')


        # fit e avaliacao pela randomized search
        model_trained, tempo_total, f1, report = fit_e_avalia(random_search)
            
        print('---')
        resultados = model_trained.cv_results_

        for params, score in zip(resultados['params'], resultados['mean_test_score']):
            print(f"Parâmetros: {params}, Score: {score}")
        print('---')    
            
        # melhor metrica na random search
        score_random_search = model_trained.best_score_
        score_random_search *= 100
        score_random_search = round(score_random_search,2)
        print(f'Melhor F1 na Random Search: {score_random_search}%')
        
        # melhores parametros encontrados
        print('Melhores parâmetros encontrados:')
        print(model_trained.best_params_)

        
        # acuracia da predicao
        print(f'F1 macro = {f1}%')

        # classification report
        print(report)

        # Avaliação Ivandre
        pipeline = Pipeline([
                        ('vect', TfidfVectorizer()),
                        ('scaling', scaler), 
                        ('selection', selection),
                        ('estimator', model)
                        ])
    
        pipeline = pipeline.set_params(**model_trained.best_params_)
    
        print(f'get_params: {pipeline.get_params}')
        
        inicio_aval_iv = datetime.datetime.now()
        acc_iv = cross_val_score(pipeline, X, y, scoring='accuracy', cv=10, n_jobs=2).mean()
        acc_iv *= 100
        acc_iv = round(acc_iv,2)
        fim_aval_iv = datetime.datetime.now()
        tempo_aval_iv = fim_aval_iv - inicio_aval_iv
        print(f'Duração da Avaliação Ivandre: {tempo_aval_iv}')
    
        print(f'Acurácia Ivandre = {acc_iv}%')
                
        
        print('----------------------------------------------')
        
        # --- Escrita em memória secundária ---

        # Nova linha que sera adicionada
        nova_linha = {'modelo': model, 'vect': TfidfVectorizer(analyzer='word'),
                    'scaling': scaler,
                      'duracao_random_search': tempo_total,
                      'qnt_iteracoes': iteracoes,
                      'f1_randsearch': f'{score_random_search}%',
                      'f1_pred': f'{f1}%', 'class_report': report,
                       'duracao_aval_iv': tempo_aval_iv,
                       'acc_aval_iv': f'{acc_iv}%',
                       'melhores_parametros': str(model_trained.best_params_)}
    
        # Cria um novo DataFrame com a nova linha
        nova_linha_resultados = pd.DataFrame([nova_linha])
    
        # Concatena o novo DataFrame com o DataFrame existente
        df_resultados = pd.concat([df_resultados, nova_linha_resultados], ignore_index=True)

        # salvamento do dataframe de resultados apos os testes terem terminado
        df_resultados.to_csv(nome_arquivo, index=False)


print('Fim dos testes')

Fim dos testes


In [16]:
modelos = [MultinomialNB()]
iteracoes = 1000
compara(iteracoes, modelos, 'nb-pos-tag-' + str(iteracoes) + '.csv')

Modelo: MultinomialNB()
Scaler: MaxAbsScaler()
Duração da Random Search: 0:55:19.442265
---
Parâmetros: {'vect__ngram_range': (1, 2), 'selection__percentile': 33, 'estimator__fit_prior': True, 'estimator__alpha': 50}, Score: 0.15300838583285453
Parâmetros: {'vect__ngram_range': (1, 3), 'selection__percentile': 33, 'estimator__fit_prior': True, 'estimator__alpha': 50}, Score: 0.13615451682506668
Parâmetros: {'vect__ngram_range': (1, 4), 'selection__percentile': 33, 'estimator__fit_prior': True, 'estimator__alpha': 50}, Score: 0.13392857142857142
Parâmetros: {'vect__ngram_range': (2, 3), 'selection__percentile': 33, 'estimator__fit_prior': True, 'estimator__alpha': 50}, Score: 0.14076988737843688
Parâmetros: {'vect__ngram_range': (2, 4), 'selection__percentile': 33, 'estimator__fit_prior': True, 'estimator__alpha': 50}, Score: 0.134187300610838
Parâmetros: {'vect__ngram_range': (3, 4), 'selection__percentile': 33, 'estimator__fit_prior': True, 'estimator__alpha': 50}, Score: 0.1390781126